# XGBoost Modeling

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

C:\Users\zasz\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\zasz\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# 1.1 Load Data

In [2]:
train = pd.read_csv("train_engineered.csv")
val = pd.read_csv("val_engineered.csv")

# 1.2 PreProcessing

In [3]:
X_train, y_train = train.drop("blueWins", axis=1), train.blueWins
X_val, y_val = val.drop("blueWins", axis=1), val.blueWins

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

pca = PCA(n_components=5, random_state=42)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)

# 2.1 Hyper Parameter Tuning

In [4]:
clf = XGBClassifier()
params = {"max_depth":[3, 4, 5]}

gs = GridSearchCV(clf, params)
gs.fit(X_train, y_train)

C:\Users\zasz\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None,
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, 

In [5]:
gs.best_params_

{'max_depth': 3}

In [6]:
model = gs.best_estimator_

# 3.1 Evaluation

In [7]:
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

print("training score:", accuracy_score(y_train, train_pred))
print("validation score:", accuracy_score(y_val, val_pred))

training score: 0.791036717062635
validation score: 0.7091203453858608
